# TimescaleDB runbook

In [ ]:
%%bash
docker run --name time -d -e POSTGRES_HOST_AUTH_METHOD=trust -p 5432:5432 timescale/timescaledb:2.16.1-pg16
sleep 4


In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE EXTENSION IF NOT EXISTS timescaledb;

"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE TABLE transactions (
   time TIMESTAMPTZ,
   block_id INT,
   hash TEXT,
   size INT,
   weight INT,
   is_coinbase BOOLEAN,
   output_total BIGINT,
   output_total_usd DOUBLE PRECISION,
   fee BIGINT,
   fee_usd DOUBLE PRECISION,
   details JSONB
);

SELECT create_hypertable('transactions', by_range('time'));
"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE INDEX hash_idx ON public.transactions USING HASH (hash);
CREATE INDEX block_idx ON public.transactions (block_id);

"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "\COPY transactions FROM 'tutorial_bitcoin_sample.csv' CSV HEADER;"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

WITH recent_blocks AS (
    SELECT block_id FROM transactions
    WHERE is_coinbase IS TRUE
    ORDER BY time DESC
    LIMIT 5
)
SELECT
    t.block_id, count(*) AS transaction_count,
    SUM(weight) AS block_weight,
    SUM(output_total_usd) AS block_value_usd
FROM transactions t
INNER JOIN recent_blocks b ON b.block_id = t.block_id
WHERE is_coinbase IS NOT TRUE
GROUP BY t.block_id;
"

In [ ]:
%%bash
docker stop time
docker rm time

In [ ]:
%%bash
docker run --name slowtime -d -e POSTGRES_HOST_AUTH_METHOD=trust -p 5432:5432 postgres:16.2
sleep 2

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

CREATE TABLE transactions (
   time TIMESTAMPTZ,
   block_id INT,
   hash TEXT,
   size INT,
   weight INT,
   is_coinbase BOOLEAN,
   output_total BIGINT,
   output_total_usd DOUBLE PRECISION,
   fee BIGINT,
   fee_usd DOUBLE PRECISION,
   details JSONB
);

CREATE INDEX hash_idx ON public.transactions USING HASH (hash);
CREATE INDEX block_idx ON public.transactions (block_id);
"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "\COPY transactions FROM 'tutorial_bitcoin_sample.csv' CSV HEADER;"

In [ ]:
%%bash
psql -h localhost -U postgres -d postgres -c "

WITH recent_blocks AS (
    SELECT block_id FROM transactions
    WHERE is_coinbase IS TRUE
    ORDER BY time DESC
    LIMIT 5
)
SELECT
    t.block_id, count(*) AS transaction_count,
    SUM(weight) AS block_weight,
    SUM(output_total_usd) AS block_value_usd
FROM transactions t
INNER JOIN recent_blocks b ON b.block_id = t.block_id
WHERE is_coinbase IS NOT TRUE
GROUP BY t.block_id;
"

In [ ]:
%%bash
docker stop slowtime
docker rm slowtime